In [1]:
import os
import cv2
import random
import numpy as np
from albumentations import HorizontalFlip, Rotate

In [ ]:
# 分割 T1 T2
archive_path = '../input/train/'
T2_path = '../input/T2Train/'
T1_path = '../input/T1Train/'

dirs = ['yes/', 'no/']
for dir in dirs:
  archive = os.listdir(archive_path + dir)
  T2 = os.listdir(T2_path + dir)
  for filename in archive:
    if filename not in T2:
      image = cv2.imread(archive_path + dir + filename, cv2.IMREAD_COLOR)
      cv2.imwrite(T1_path + dir + filename, image)

In [39]:
def seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed()

In [ ]:
all_path = '../input/train/'
t1_path = '../input/T1Train/'
t2_path = '../input/T2Train/'
t1_dst_path = '../input/T1TrainSameSize/'
t2_dst_path = '../input/T2TrainSameSize/'
path_lst = [t1_path, t2_path]
dst_path_lst = [t1_dst_path, t2_dst_path]

dirs = ['yes/', 'no/']
for i in range(len(path_lst)):
    path = path_lst[i]
    dst_path = dst_path_lst[i]
    for dir in dirs:
        num_all = len(os.listdir(all_path + dir))
        num = len(os.listdir(path + dir))
        lst = []
        for filename in os.listdir(path + dir):
            lst.append(filename)
            img = cv2.imread(path + dir + filename, cv2.IMREAD_COLOR)
            cv2.imwrite(dst_path + dir + filename, img)
        lst = np.array(lst)
        np.random.shuffle(lst)

        flag = False
        for type in ['h_', 'r10_', 'r20_', 'r30_']:
            if flag:
                break
            
            for j in range(num):
                if len(os.listdir(dst_path + dir)) == num_all:
                    flag = True
                    break
                filename = lst[j]
                img = cv2.imread(path + dir + filename, cv2.IMREAD_COLOR)
                if type == 'h_':
                    image = HorizontalFlip(p=1)(image=img)['image']
                elif type == 'r10_':
                    image = Rotate(p=1)(image=img, limit=(10, 11), border_mode=cv2.BORDER_CONSTANT, value=0)['image']
                elif type == 'r20_':
                    image = Rotate(p=1)(image=img, limit=(20, 21), border_mode=cv2.BORDER_CONSTANT, value=0)['image']
                elif type == 'r30_':
                    image = Rotate(p=1)(image=img, limit=(30, 31), border_mode=cv2.BORDER_CONSTANT, value=0)['image']
                cv2.imwrite(dst_path + dir + type + filename, image)


In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [ ]:
class CFG:
    n_fold = 4

In [ ]:
T1_PATH = '../input/T1Train/'
T2_PATH = '../input/T2Train/'

folds = {'filename': [], 'label': [], 'T1': []}
for filename in os.listdir(T1_PATH + 'yes/'):
  folds['filename'].append(filename)
  folds['label'].append(1)
  folds['T1'].append(1)
for filename in os.listdir(T1_PATH + 'no/'):
  folds['filename'].append(filename)
  folds['label'].append(0)
  folds['T1'].append(1)
for filename in os.listdir(T2_PATH + 'yes/'):
  folds['filename'].append(filename)
  folds['label'].append(1)
  folds['T1'].append(0)
for filename in os.listdir(T2_PATH + 'no/'):
  folds['filename'].append(filename)
  folds['label'].append(0)
  folds['T1'].append(0)

folds = pd.DataFrame(folds)
Fold = StratifiedKFold(n_splits=CFG.n_fold)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds['label'])):
  folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)

display(folds.groupby('fold').size())
display(folds)

folds.to_csv('../input/RealTrain/data.csv', index=False, sep=',')

In [59]:

folds = pd.read_csv('../input/RealTrain/data.csv')
T1_idx = 2
T1 = folds['T1'].values

np.random.seed(42)
np.random.shuffle(T1)

folds['T1'] = T1

display(folds.groupby('fold').size())
display(folds)

folds.to_csv('../input/RealTrainRandom/data.csv', index=False, sep=',')

[1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 0 0 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1
 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0
 1 1 0 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1
 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1
 1 1 1 0 1]


fold
0    58
1    57
2    56
3    56
dtype: int64

,filename,label,T1,fold
0,Y1.jpg,1,1,0
1,Y10.jpg,1,1,0
2,Y101.jpg,1,1,0
3,Y103.jpg,1,1,0
4,Y104.jpg,1,1,0
...,...,...,...,...
222,No12.jpg,0,1,3
223,No14.jpg,0,1,3
224,No15.jpg,0,1,3
225,No21.jpg,0,0,3
